# Question 2. IWAL algorithm implementation (50 points)

The purpose of this question is to implement Importance Weighted Active Learning (IWAL) algorithm. For this question, you will not use modAL, but instead will implement IWAL routine from scratch using scikit-learn, NumPy and native Python. 

In this question, we will use a simple synthetic dataset for a binary classification problem. Each data point has only 2 features. The dataset is provided in 2 files -- “data_iwal.npy”, which contains features and “labels_iwal.npy”, which contains labels. 

For simplicity, you will implement bootstrapping rejection sampling subroutine with logistic regression and hinge loss.   

𝐂𝐨𝐦𝐩𝐥𝐞𝐭𝐞 𝐭𝐡𝐞 𝐜𝐨𝐝𝐞 𝐮𝐧𝐝𝐞𝐫 ###𝐓𝐎 𝐃𝐎 𝐢𝐧 𝐞𝐚𝐜𝐡 𝐜𝐞𝐥𝐥 𝐚𝐧𝐝 𝐩𝐫𝐨𝐝𝐮𝐜𝐞 𝐭𝐡𝐞 𝐫𝐞𝐪𝐮𝐢𝐫𝐞𝐝 𝐩𝐥𝐨𝐭𝐬.  Feel free to define any helper functions as you see fit. You may import and use any modules in scikit-learn and NumPy to help with your implementations.

## Imports

Here we import necessary modules. Feel free to add something else here if you need it!

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import hinge_loss, log_loss
from sklearn.linear_model import LogisticRegression

# my package
from packages.iwal import iwal_functions

## Reading data

Here we read the data and split it into train and test datasets. Train will be used to train our classification model and test will be used to validate the performance, monitor overfitting and compare the results of the model trained with Active Learning with the ones of the model trained from scratch. We set aside 1/3 of the dataset for validation.

In [13]:
X = np.load("data/q2/data_iwal.npy")
y = np.load("data/q2/labels_iwal.npy")

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [25]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X[0:2])
print(y[0:2])

(134, 2)
(134,)
(66, 2)
(66,)
[[2.59193175 1.14706863]
 [1.7756532  1.15670278]]
[1 1]


In [50]:
from scipy.stats import bernoulli
bernoulli.rvs(1.0)

1

## Part 2.1
Type your answers for the theoretical questions below.

1. What is the idea behind IWAL algorithm?

**Your answer goes here**

2. What are the assumptions made for the IWAL algorithm?

**Your answer goes here**

3. What are the pros and cons of IWAL algorithm?

**Your answer goes here**

## Part 2.2 Implement IWAL algorithm

In this part you will implement a functiom that performs a single query of Algorithm 1 IWAL (subroutine rejection-sampling) from the paper. Below is the function description that you can follow in your implementation.

### Equations

$h_t = \underset{h \in H}{argmin}\sum_{(x,y,c)\in S_t}c \cdot l(h(x),y)$

In [ ]:
### TODO: Implement Algorithm 1 from the paper

In [11]:
lr = LogisticRegression().fit()
lr.predict([1])

TypeError: fit() missing 2 required positional arguments: 'X' and 'y'

## Part 2.3 Implement bootstrapping rejection sampling subroutine
In this part you will implement bootstrapping rejection sampling subroutine from the paper, section 7.2

### Equation
$p_t = p_{min}+(1-p_{min})[\underset{y;h_i,h_j \in H}{max}L(h_i(x),y)-L(h_j(x),y)]$

where $p_{min}$ is a lower bound on the sampling probability

In [6]:
def loss_difference(y_true, y_pred_i, y_pred_j, labels, loss):
    '''
    Calculates the difference in loss.
    '''
    loss_i = loss(y_true, y_pred_i,labels=labels)
    loss_j = loss(y_true, y_pred_j,labels=labels)
    return loss_i - loss_j 


In [9]:
print('test loss_difference()')
test_y_true = np.asarray([0,1])
test_y_pred_i = np.asarray([1,1])
test_y_pred_j = np.asarray([0,1])
loss_difference(test_y_true, test_y_pred_i,test_y_pred_j,[0,1],hinge_loss)


test loss_difference()


0.5

In [ ]:
#?? don't use loss parameter
def bootstrap(x, H, labels, loss, p_min=0.1):
    '''
    This function implements bootstrap rejection threshold 
    subroutine.
    
    Args:
        x: array-like object of features for currently considered 
        sample.
        
        H: list of hypothesis (scikit-learn objects) that are used 
        in voting.
        
        labels: list of possible labels for the problem. If binary 
        classification, labels=[0, 1].
        
        loss: python function, accepts (y_true,y_pred,labels=labels) and returns
        loss of prediction.
        
        p_min: minimum threshold for the probability.
        
    Returns:
        p_t: probability of requesting the label for x, which is 
        equal to:
            p_min + (1 - p_min)(max_{y, h_i, h_j} L(h_i(x), y) - L(h_j(x), y))
    '''
    max_value = -10000
    
    for i in range(len(H)):
        for j in range(len(H)):
            for label in labels:
                
                # calculate loss difference between models
                y_true = np.full(shape=len(x),fill_value=label,dtype=np.int)
                y_pred_i = H[i].predict(x)
                y_pred_j = H[j].predict(x)
                
                curr = loss_difference(y_true, y_pred_i, y_pred_j, labels, loss)
                
                # update max
                if curr > max_value:
                    max_value = curr
                         
    return p_min + (1 - p_min) * max_value

In [ ]:
print('test bootstrap()')


## Part 2.4 Organize all implemented parts into a single pipeline

Now you implemented all part of IWAL algorithm with bootstrap rejection sampling and can organize it into a pipeline

In [ ]:
history = {}
losses = []
n_initial = 10
n_h = 10

X_training, y_training = X_train[:n_initial], y_train[:n_initial]

#Initialization: initialize history and H
## TODO: your code here

# Create n_h classifiers and train them on bootstrapped data (data is sampled with replacement)
## TODO: your code goes here
    
# Perform queries and record loss
n_query = 50
for t in n_query:
    ### TODO: your code goes here
    log_loss(y_test, h_t.predict_proba(X_test)) 

In [ ]:
plt.plot(losses)

## Part 2.5 Compare results of Active Learning vs No Active Learning

In this part you need to create object of the same scikit learning class and train it on randomly selected subset of data points and compare results of 2 classifiers. Comment on your observations

In [ ]:
# Compare to no Active Learning setting

## TODO: your code goes here